# Feeding data to a production model

This notebook is part of the [watsonx.governance Level 4 Proof of Experience (PoX) hands-on lab](watsonx.governance Level 4 Proof of Experience (PoX) hands-on lab). It will query a deployed production model

In [ ]:
USERNAME = "complianceofficer"
PASSWORD = "passw0rd"
CPD_URL = "https://cpd-cpd.apps.___________.cloud.techzone.ibm.com"
INSTANCE_ID = "openshift"
VERSION = "4.8"
DEPLOYMENT_ID = ""
SPACE_GUID = ""
RECORDS_TO_SCORE = 100

### Authenticate with the watsonx service running in the environment

In [ ]:
from ibm_watsonx_ai import APIClient

wml_credentials = {
    "username": USERNAME,
    "password": PASSWORD,
    "url": CPD_URL,
    "instance_id": 'openshift',
    "version": '4.8'
}

client = APIClient(wml_credentials)

### Set the client to use the deployment space for the production model

In [ ]:
client.set.default_space(SPACE_GUID)

### Import data to be scored

In [ ]:
import pandas as pd

feed_dataframe = pd.read_csv("https://raw.githubusercontent.com/CloudPak-Outcomes/Outcomes-Projects/main/watsonx-governance-l4/data/hiring_training_data.csv")
feed_dataframe.head()

### Remove the label column from the data

In [ ]:
feed_dataframe.drop(['HIRED'], axis=1, inplace=True)

### Identify the features and values, and construct the payload data

In [ ]:
features = feed_dataframe.columns.values
values = feed_dataframe.sample(n=RECORDS_TO_SCORE).round().astype(int).values

scoring_payload = {client.deployments.ScoringMetaNames.INPUT_DATA:
    [{'fields':
        features,
        'values': values
    }]
}

### Send the scoring payload to the model for scoring

In [ ]:
predictions = client.deployments.score(DEPLOYMENT_ID, scoring_payload)

### Display the model predictions

If you have configured monitoring for the model, the input and output data here will be automatically recorded in the watsonx monitoring service datamart, and you will be able to calculate drift and fairness using the data.

In [ ]:
predictions